### Ingesta de datos del proyecto PI_ML_OPS cohorte DTS-12 
#### https://github.com/Raftxo-MX/PI_ML_OPS.git

Importamos las librerías necesarias en el proceso ETL


In [14]:
import pandas as pd
import numpy as np
import ast
import re
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')

Cargamos los datos CSV en dataframes:

In [2]:
movies_df = pd.read_csv('./data/movies_dataset.csv')
credits_df = pd.read_csv('./data/credits.csv')

C:\Users\raftx\AppData\Local\Temp\ipykernel_23316\3250179558.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('./data/movies_dataset.csv')


Convertimos los valores de la columna 'budget' de string a integer

In [ ]:
movies_df['budget'] = movies_df['budget'].astype(int)
# da errores de momento, hay que encontrar valores chungueras

Primero vamos a buscar filas con valores nulos en 'id'

In [86]:
mask = movies_df['id'].isnull()
issues = movies_df.loc[mask]
print(issues)

Empty DataFrame
Columns: [adult, belongs_to_collection, budget, genres, homepage, id, imdb_id, original_language, original_title, overview, popularity, poster_path, production_companies, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, title, video, vote_average, vote_count]
Index: []

[0 rows x 24 columns]


También quiero saber los valores nulos en 'title'.
Hay 6 filas con los datos corridos con valores NaN en columnas importantes para la API

In [85]:
mask = movies_df['title'].isnull()
issues = movies_df.loc[mask]
print(issues)
# en este query encontramos un patrón de datos traslapados

                                                   adult  \
19729                                              False   
19730                                 - Written by Ørnås   
29502                                              False   
29503   Rune Balot goes to a casino connected to the ...   
35586                                              False   
35587   Avalanche Sharks tells the story of a bikini ...   

                                   belongs_to_collection  \
19729                                                NaN   
19730                                           0.065736   
29502  {'id': 122661, 'name': 'Mardock Scramble Colle...   
29503                                           1.931659   
35586                                                NaN   
35587                                           2.185485   

                                 budget  \
19729                                 0   
19730  /ff9qCepilowshEtG2GYWwzt2bs4.jpg   
29502                        

In [94]:
mask = movies_df['id'].str.len() > 8
issues = movies_df.loc[mask]

In [95]:
print(issues)

                                                   adult  \
19730                                 - Written by Ørnås   
29503   Rune Balot goes to a casino connected to the ...   
35587   Avalanche Sharks tells the story of a bikini ...   

      belongs_to_collection                            budget  \
19730              0.065736  /ff9qCepilowshEtG2GYWwzt2bs4.jpg   
29503              1.931659  /zV8bHuSL6WXoD6FWogP9j4x80bL.jpg   
35587              2.185485  /zaSf5OG7V8X8gqFvly88zDdRm46.jpg   

                                                  genres  \
19730  [{'name': 'Carousel Productions', 'id': 11176}...   
29503  [{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...   
35587  [{'name': 'Odyssey Media', 'id': 17161}, {'nam...   

                                                homepage          id imdb_id  \
19730  [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...  1997-08-20       0   
29503  [{'iso_3166_1': 'US', 'name': 'United States o...  2012-09-29       0   
35587           [

In [91]:
mask = movies_df['budget'].str.contains('ff9qCepilowshEtG2GYWwzt2bs4')
issues = movies_df.loc[mask]
print(issues['id'])

# aquí encontramos una fila que nos salía también problemática al comprobar los 'id'


19730    1997-08-20
Name: id, dtype: object


In [93]:
mask = movies_df['id'].str.len() > 8
issues = movies_df.loc[mask]
print(issues['id'])

19730    1997-08-20
29503    2012-09-29
35587    2014-01-01
Name: id, dtype: object


### Hay filas con datos traslapados
## 19729 y 19730 
## 19502 y 29503
## 35586 y 35587

In [104]:
movies_df.loc[19729]

adult                                                                False
belongs_to_collection                                                  NaN
budget                                                                   0
genres                   [{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...
homepage                                                               NaN
id                                                                   82663
imdb_id                                                          tt0113002
original_language                                                       en
original_title                                                Midnight Man
overview                 British soldiers force a recently captured IRA...
popularity                                                        0.065736
poster_path                               /ff9qCepilowshEtG2GYWwzt2bs4.jpg
production_companies     [{'name': 'Carousel Productions', 'id': 11176}...
production_countries     

In [105]:
movies_df.loc[19730]

adult                                                   - Written by Ørnås
belongs_to_collection                                             0.065736
budget                                    /ff9qCepilowshEtG2GYWwzt2bs4.jpg
genres                   [{'name': 'Carousel Productions', 'id': 11176}...
homepage                 [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...
id                                                              1997-08-20
imdb_id                                                                  0
original_language                                                    104.0
original_title                    [{'iso_639_1': 'en', 'name': 'English'}]
overview                                                          Released
popularity                                                             NaN
poster_path                                                   Midnight Man
production_companies                                                 False
production_countries     

In [98]:
# una manera rudimentaria sería ir moviendo uno a uno los datos, por ejemplo:
# Mover 'belongs_to_collection' de 19730 a 'popularity' de 19729
movies_df.loc[19729, 'popularity'] = movies_df.loc[19730, 'belongs_to_collection']

# Mover 'budget' de 19730 a 'poster_path' de 19729
movies_df.loc[19729, 'poster_path'] = movies_df.loc[19730, 'budget']

In [102]:
# otra manera sería hacerlo con una función con un loop simple:
def unlap(row):
    for i in range(1,15):
        j=i+9
        movies_df.iloc[row-1,j] = movies_df.iloc[row,i]

In [103]:
# Vamos a probar la función con las filas 19730 y la anterior
unlap(19730)
# todo perfecto, ya tenemos la fila 19729 bien bonita, con todos los datos faltantes
# más adelante borraremos la fila 19730 que ya no nos sirve para nada

In [106]:
# procedemos a "destraspolar" las otras dos filas traspoladas
unlap(29503)
unlap(35587)

In [107]:
# comprobamos las filas 29502 y 35586
movies_df.loc[29502]

adult                                                                False
belongs_to_collection    {'id': 122661, 'name': 'Mardock Scramble Colle...
budget                                                                   0
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 878, ...
homepage                                     http://m-scramble.jp/exhaust/
id                                                                  122662
imdb_id                                                          tt2423504
original_language                                                       ja
original_title                                            マルドゥック・スクランブル 排気
overview                        Third film of the Mardock Scramble series.
popularity                                                        1.931659
poster_path                               /zV8bHuSL6WXoD6FWogP9j4x80bL.jpg
production_companies     [{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...
production_countries     

In [108]:
movies_df.loc[35586]

adult                                                                False
belongs_to_collection                                                  NaN
budget                                                                   0
genres                   [{'id': 10770, 'name': 'TV Movie'}, {'id': 28,...
homepage                                                               NaN
id                                                                  249260
imdb_id                                                          tt2622826
original_language                                                       en
original_title                                            Avalanche Sharks
overview                 A group of skiers are terrorized during spring...
popularity                                                        2.185485
poster_path                               /zaSf5OG7V8X8gqFvly88zDdRm46.jpg
production_companies     [{'name': 'Odyssey Media', 'id': 17161}, {'nam...
production_countries     

In [109]:
non_numeric = movies_df['budget'].isna().sum()
print(f"{non_numeric} valores no numéricos")

0 valores no numéricos


In [110]:
types = movies_df['budget'].apply(type).value_counts()
print(types)

budget
<class 'str'>    45466
Name: count, dtype: int64


In [56]:
numeric = movies_df['budget'].str.isnumeric()

In [117]:
print(numeric.sample(28))

38233    True
24321    True
42343    True
6473     True
18892    True
7536     True
33725    True
40568    True
20080    True
30964    True
21087    True
33284    True
8171     True
34386    True
44806    True
8852     True
5891     True
473      True
6188     True
33457    True
37839    True
32371    True
28583    True
43612    True
41347    True
33901    True
42654    True
4507     True
Name: budget, dtype: bool


In [120]:
# eliminamos las filas problemáticas 35587, 29503 y 19730
problem_rows = [35587, 29503, 19730]
movies_df = movies_df.drop(problem_rows)

Ahora sí que podemos ya convertir los valores de la columna 'budget' de string a integer

In [121]:
movies_df['budget'] = movies_df['budget'].astype(int)
# convertidos los valores 'budget' de string a integer

In [122]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45463 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   adult                  45463 non-null  object
 1   belongs_to_collection  4491 non-null   object
 2   budget                 45463 non-null  int32 
 3   genres                 45463 non-null  object
 4   homepage               7779 non-null   object
 5   id                     45463 non-null  object
 6   imdb_id                45446 non-null  object
 7   original_language      45452 non-null  object
 8   original_title         45463 non-null  object
 9   overview               44509 non-null  object
 10  popularity             45463 non-null  object
 11  poster_path            45080 non-null  object
 12  production_companies   45463 non-null  object
 13  production_countries   45463 non-null  object
 14  release_date           45379 non-null  object
 15  revenue                4

Sería un buen momento ahora para eliminar las columnas que no serán utilizadas según reza el readme.md del proyecto:
- video,imdb_id,adult,original_title,poster_path y homepage

In [124]:
# Definimos las columnas a dropear
columns_to_drop = ['video','imdb_id','adult','original_title','poster_path','homepage']
# Dropeamos con el argumento axis=1 para indicar que queremos eliminar columnas
movies_df = movies_df.drop(columns_to_drop,axis=1)

In [125]:
# comprobamos el shape
print(movies_df.shape)

(45463, 18)


Ahora sería un buen momento para eliminar duplicidades del 'id'.

In [127]:
# ver filas con 'id' repetido
ids_duplicated = movies_df['id'].duplicated()
duplicated_indices = movies_df[ids_duplicated].index
duplicated_rows = movies_df.loc[duplicated_indices]
print(duplicated_rows[['id','title']])
# se han encontrado 30 id repes pero algunos están repes más de 2 veces
# así que las filas a borrar serán más...

           id                                        title
1465   105045                                  The Promise
9165     5511                                  Le Samouraï
9327    23305                                  The Warrior
12066   14788                                       Bubble
13375  141971                                     Blackout
15074   22649                           A Farewell to Arms
15765   13209                                      Offside
16764  141971                                     Blackout
20843   77221                                   Black Gold
20899  109962                              Rich and Famous
21116   84198                         A Place at the Table
21165  119916                                  The Tempest
21854  152795                                 The Congress
22151   18440                             Days of Darkness
23044   25541                                  Brotherhood
23534  110428                         Camille Claudel 19

In [138]:
# eliminar filas con 'id' repetido con .drop_duplicates
# subset identifica los valores duplicados en la columna especificada (en este caso, 'id')
# keep='first' conserva la primera fila de las duplicadas y elimina el resto
# por defecto es así, no es obligatorio poner keep='first'
# otra posibilidad sería poner keep='last'
movies_df = movies_df.drop_duplicates(subset='id', keep='first')

In [139]:
movies_df.shape

(45433, 18)

En este punto voy a hacer una copia de seguridad en formato .pickle

In [147]:
# guardamos un backup de movies_df
movies_df.to_pickle('./data/movies_df.pickle')



In [150]:
movies_df.dtypes

belongs_to_collection    object
budget                    int32
genres                   object
id                       object
original_language        object
overview                 object
popularity               object
production_companies     object
production_countries     object
release_date             object
revenue                  object
runtime                  object
spoken_languages         object
status                   object
tagline                  object
title                    object
vote_average             object
vote_count               object
dtype: object

Vamos a revisar la columna 'revenue' para después junto con la de 'budget' buscar los nulos y rellenarlos con valor 0.

In [166]:
# enumeramos diverentes tipos de datos en la columna 'revenue'
types = movies_df['revenue'].apply(type).value_counts()
print(types)

revenue
<class 'float'>    45430
<class 'str'>          3
Name: count, dtype: int64


In [153]:
# buscamos las 3 filas de tipo string (que no sean float)
not_float = movies_df['revenue'].apply(type) != float

In [154]:
# obtenemos los índices de esas filas:
non_float_idx = movies_df[not_float].index

In [155]:
# y seleccionamos solo esas filas
non_float_rows = movies_df.loc[non_float_idx]

In [160]:
print(non_float_rows)

                                   belongs_to_collection  budget  \
19729                                                NaN       0   
29502  {'id': 122661, 'name': 'Mardock Scramble Colle...       0   
35586                                                NaN       0   

                                                  genres      id  \
19729  [{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...   82663   
29502  [{'id': 16, 'name': 'Animation'}, {'id': 878, ...  122662   
35586  [{'id': 10770, 'name': 'TV Movie'}, {'id': 28,...  249260   

      original_language                                           overview  \
19729                en  British soldiers force a recently captured IRA...   
29502                ja         Third film of the Mardock Scramble series.   
35586                en  A group of skiers are terrorized during spring...   

      popularity                               production_companies  \
19729   0.065736  [{'name': 'Carousel Productions', 'id': 11176}...   

In [210]:
movies_df.dtypes

belongs_to_collection     object
budget                     int32
genres                    object
id                        object
original_language         object
overview                  object
popularity                object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                   object
spoken_languages          object
status                    object
tagline                   object
title                     object
vote_average              object
vote_count                object
return                   float64
dtype: object

In [167]:
# Reemplazamos '0' por 0 
non_float_rows['revenue'] = non_float_rows['revenue'].replace({'0': 0})

In [168]:
# Convertir el tipo a float  
non_float_rows['revenue'] = non_float_rows['revenue'].astype(float)

In [171]:
movies_df['revenue'] = movies_df['revenue'].astype(float)
# ahora sí, convertidos los valores 'revenue' de string a float

In [172]:
# enumeramos diverentes tipos de datos en la columna 'revenue'
types = movies_df['revenue'].apply(type).value_counts()
print(types)

revenue
<class 'float'>    45433
Name: count, dtype: int64


In [174]:
# Encontrar NaN
null_values = movies_df['revenue'].isna()  

# Obtener índices de las filas con NaN
null_idx = movies_df[null_values].index

# Seleccionar esas filas
null_rows = movies_df.loc[null_idx]

In [175]:
print(null_values)
print(null_rows)

0        False
1        False
2        False
3        False
4        False
         ...  
45461    False
45462    False
45463    False
45464    False
45465    False
Name: revenue, Length: 45433, dtype: bool
Empty DataFrame
Columns: [belongs_to_collection, budget, genres, id, original_language, overview, popularity, production_companies, production_countries, release_date, revenue, runtime, spoken_languages, status, tagline, title, vote_average, vote_count]
Index: []


In [208]:
print(movies_df['budget'].sample(28))

2179     12000000
28185           0
28428           0
23152         200
623             0
19126           0
29675           0
28339           0
24780           0
35438           0
33639      300000
26332           0
13341           0
41664           0
7956      3000000
27216           0
22678           0
5791            0
3707      1500000
35469           0
18117           0
4060            0
34135           0
12521    25000000
8277            0
1918     10000000
5917            0
9290            0
Name: budget, dtype: int32


In [196]:
print((movies_df['revenue']==0).sum())
print((movies_df['budget']==0).sum())

38035
36553


In [197]:
print((movies_df['revenue']!=0).sum())
print((movies_df['budget']!=0).sum())

7398
8880


Crear columna `return` calculando el retorno de inversión = revenue / budget

In [209]:
# Creamos columna nueva con los valores a 0 inicialmente
movies_df['return'] = 0

# Calcular return donde es posible (budget > 0)
# Cuando budget == 0, return se quedará en 0
valid = movies_df['budget'] > 0
movies_df.loc[valid, 'return'] = movies_df.loc[valid, 'revenue'] / movies_df.loc[valid, 'budget']

# Cuando revenue es NaN, return es también NaN
# movies_df['return'].fillna(0, inplace=True)

In [249]:
print(movies_df['return'].sample(28))

26515    0.000000
23070    0.000000
25960    0.000000
40655    0.000000
6849     6.870438
1744     0.000000
41122    0.000000
33716    0.000000
37324    0.000000
36493    0.000000
10992    0.000000
30068    0.000000
5386     0.000000
38288    0.000000
23455    0.000000
3590     0.000000
44232    0.000000
35261    0.000000
2298     9.692701
38720    0.000000
24615    0.000000
806      4.476923
707      0.000000
15253    0.000000
17753    0.000000
3268     0.000000
39893    0.000000
41002    0.000000
Name: return, dtype: float64


In [250]:
movies_df.loc[6849]

belongs_to_collection                                                  NaN
budget                                                             3500000
genres                   [{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...
id                                                                   11850
original_language                                                       en
overview                 The first remake of the paranoid infiltration ...
popularity                                                        5.219641
production_companies     [{'name': 'United Artists', 'id': 60}, {'name'...
production_countries     [{'iso_3166_1': 'US', 'name': 'United States o...
release_date                                                    1978-12-19
revenue                                                         24046533.0
runtime                                                              115.0
spoken_languages         [{'iso_639_1': 'cs', 'name': 'Český'}, {'iso_6...
status                   

___
Punto de backup 2


In [251]:
movies_df.to_pickle('./data/movies_df2.pickle')

In [252]:
# Eliminamos los valores nulos de la columna 'release_date'
movies_df.dropna(subset=['release_date'], inplace=True)

In [275]:
# Convertir las fechas a datetime y formatear a AAAA-MM-DD
movies_df['release_date'] = pd.to_datetime(movies_df['release_date']).dt.strftime('%Y-%m-%d')

In [282]:
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'])

In [283]:
# Crear columna 'release_year'
movies_df['release_year'] = movies_df['release_date'].dt.year

In [290]:
movies_df.dtypes

belongs_to_collection            object
budget                            int32
genres                           object
id                               object
original_language                object
overview                         object
popularity                       object
production_companies             object
production_countries             object
release_date             datetime64[ns]
revenue                         float64
runtime                          object
spoken_languages                 object
status                           object
tagline                          object
title                            object
vote_average                     object
vote_count                       object
return                          float64
release_year                      int32
dtype: object

___
### Punto de backup 3
antes de empezar a desanidar las columnas 'belongs_to_collection','genres','production_companies','production_countries'

In [407]:
# movies_df.to_pickle('./data/movies_df3.pickle')
movies_df = pd.read_pickle('./data/movies_df3.pickle')

___

In [408]:
# Antes de desanidar etc... vamos a unir los dos datasets por la columna 'id'
# Cast 'id' to int in both DataFrames
credits_df['id'] = credits_df['id'].astype(int)  
movies_df['id'] = movies_df['id'].astype(int)

# Then merge  
df = pd.merge(credits_df, movies_df, on='id')

In [409]:
df.shape

(45392, 22)

In [416]:
df.dtypes

cast                             object
crew                             object
id                                int32
belongs_to_collection            object
budget                            int32
genres                           object
original_language                object
overview                         object
popularity                       object
production_companies             object
production_countries             object
release_date             datetime64[ns]
revenue                         float64
runtime                          object
spoken_languages                 object
status                           object
tagline                          object
title                            object
vote_average                     object
vote_count                       object
return                          float64
release_year                      int32
dtype: object

In [426]:
mask = df['id'] == 141971

df[mask]

,cast,crew,id,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year
13269,"[{'cast_id': 1, 'character': 'Pekka', 'credit_...","[{'credit_id': '52fe4ac89251416c750edd47', 'de...",141971,NaN,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",fi,Recovering from a nail gun shot to the head an...,0.411949,"[{'name': 'Filmiteollisuus Fine', 'id': 5166}]",...,0.0,108.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",Released,Which one is the first to return - memory or t...,Blackout,6.7,3.0,0.0,2008


In [421]:
# hay que volver a revisar si no tienen 'id' repes
ids_duplicated = df['id'].duplicated()
duplicated_indices = df[ids_duplicated].index
duplicated_rows = df.loc[duplicated_indices]
print(duplicated_rows[['id','title']])

           id                                             title
677    105045                                       The Promise
838    132641                                              Wife
950     22649                                A Farewell to Arms
2565    84198                              A Place at the Table
4114    10991                     Pokémon: Spell of the Unknown
4357   110428                              Camille Claudel 1915
5132    15028                                     Clockstoppers
5538    12600       Pokémon 4Ever: Celebi - Voice of the Forest
5714   109962                                   Rich and Famous
5870     4912                   Confessions of a Dangerous Mind
7350     5511                                       Le Samouraï
8074    23305                                       The Warrior
9581    69234                          The Phantom of the Opera
10425   14788                                            Bubble
11162   77221                           

In [429]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45348 entries, 0 to 45391
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   cast                   45348 non-null  object        
 1   crew                   45348 non-null  object        
 2   id                     45348 non-null  int32         
 3   belongs_to_collection  4486 non-null   object        
 4   budget                 45348 non-null  int32         
 5   genres                 45348 non-null  object        
 6   original_language      45337 non-null  object        
 7   overview               44407 non-null  object        
 8   popularity             45348 non-null  object        
 9   production_companies   45348 non-null  object        
 10  production_countries   45348 non-null  object        
 11  release_date           45348 non-null  datetime64[ns]
 12  revenue                45348 non-null  float64       
 13  runtim

In [423]:
# Procedemos a borrar los duplicados
df = df.drop_duplicates(subset='id', keep='first')
# se eliminan los 44 duplicados que aparecieron al unir ambos dataframes

___
### Punto backup 4


In [428]:
df.to_pickle('./data/df_backup4.pickle')
# df = pd.read_pickle('./data/df_backup4.pickle')

___

Vamos a tratar las columnas anidadas, sacar de ahí la información que nos interesa.

In [431]:
# import ast 
# comentarizo el import porque ya lo paso al principio para tener todos los imports juntos

# tras mucha investigación al final acabo con la biblioteca "ast"
# para convertir los valores 'string' de la columna 'crew' en lista de diccionarios.
df['crew'] = df['crew'].apply(ast.literal_eval)


# Seguidamente aplico la funcion lambda a todos los valores de 'crew' para extraer nombres de los directores
# compruebo con isinstance(x, list) que no haya valores None u otros que no sean de lista.
df['director_names'] = df['crew'].apply(lambda x: [d['name'] for d in x if d['job'] == 'Director'] if isinstance(x, list) else None)

In [446]:
df['director_names'].sample(5)

22654                [Ramon Zürcher]
32770    [Risto Orko, Leonid Gayday]
27827                   [Ken Kwapis]
10954        [Hans-Christian Schmid]
667                    [Peter Yates]
Name: director_names, dtype: object

In [447]:
df['crew'].sample(5)

12930    [{'credit_id': '54462a750e0a26633a00a27f', 'de...
38678    [{'credit_id': '52fe4bb4c3a36847f8210659', 'de...
40617    [{'credit_id': '5563401dc3a36854480023db', 'de...
14451    [{'credit_id': '52fe45089251416c91025319', 'de...
20905    [{'credit_id': '584f93a5c3a368315000490d', 'de...
Name: crew, dtype: object

In [499]:
df['production_companies'].sample(5)


37072                                                   []
15176                   [{'name': 'Alicéléo', 'id': 8799}]
29328    [{'name': 'Hong Hwa Motion Picture Company', '...
14310                                                   []
33309    [{'name': 'Onyx Films', 'id': 2203}, {'name': ...
Name: production_companies, dtype: object

In [449]:
# Renombramos la columna 'director_names' por 'directed_by' que no se nos ocurrió antes pero es mejor nombre...
df.rename(columns={'director_names':'directed_by'}, inplace=True)

In [454]:
df.columns

Index(['cast', 'crew', 'id', 'belongs_to_collection', 'budget', 'genres',
       'original_language', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'return', 'release_year', 'directed_by'],
      dtype='object')

In [453]:
# Vamos a intentar sacar los nombres de las franquicias
df['belongs_to_collection'] = df['belongs_to_collection'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [459]:
df['franchise'] = df['belongs_to_collection'].apply(
    lambda x: x['name'] if isinstance(x,dict) else None
)

___

In [502]:
# Vamos a por las productoras:
# Convertimos los valores string a dict usando ast.literal_eval()
df['production_companies'] = df['production_companies'].apply(ast.literal_eval)

# Extrayendo el nombre de la compañía en una lista 
df['produced_by'] = df['production_companies'].apply( 
    lambda x: [comp['name'] for comp in x] if isinstance(x, list) else None
)

___

In [508]:
# Ahora vamos a los países productores
# usamos ast.literal_eval
df['production_countries'] = df['production_countries'].apply(ast.literal_eval)

# Extraemos los nombres de los países en una lista en una columna nueva
df['produced_in'] = df['production_countries'].apply(
    lambda x: [pais['name'] for pais in x] if isinstance(x, list) else None
)

In [521]:
df['genres_clean'].sample(5)

15464                [Animation, Comedy]
1150     [Comedy, Crime, Drama, Romance]
21769                                 []
31861                            [Drama]
14981                      [Documentary]
Name: genres_clean, dtype: object

In [515]:
# Finalmente a por los 'genres'
# con el mismo procedimiento...
df['genres'] = df['genres'].apply(ast.literal_eval)

# Extraemos los nombres de los países en una lista en una columna nueva
df['genres_clean'] = df['genres'].apply(
    lambda x: [genero['name'] for genero in x] if isinstance(x, list) else None
)

___
### Punto de backup 5


In [3]:
# df.to_pickle('./data/df_backup5.pickle')

# cuando empiezas de nuevo, ejecutar primero la celda de imports al principio del notebook
df = pd.read_pickle('./data/df_backup5.pickle') # cargar el df guardado como pickle

In [5]:
df.head(1)

,cast,crew,id,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,...,title,vote_average,vote_count,return,release_year,directed_by,franchise,produced_by,produced_in,genres_clean
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]",...,Toy Story,7.7,5415.0,12.451801,1995,[John Lasseter],Toy Story Collection,[Pixar Animation Studios],[United States of America],"[Animation, Comedy, Family]"


### Borrado de columnas originales de las cuales se extrajeron los datos necesarios en nuevas columnas

In [6]:
# podemos borrar las columnas cast, crew, belongs_to_collection, 
# production_companies, production_countries, genres
# 'cast' no la usamos en ningún momento
# de 'crew' ya hemos sacado los directores en la columna 'directed_by'
# de 'belongs_to_collection' ya hemos sacado el nombre de la franquicia en 'franchise'
# de 'production_companies' se han extraido las productoras en 'produced_by'
# de 'production_countries' se han extraido los países en 'produced_in'
# de 'genres' se ha extraído la información en 'genres_clean'

df.drop(['cast', 'crew', 'belongs_to_collection', 'production_companies',
         'production_countries', 'genres'], axis=1, inplace=True)

In [11]:
print(df.columns)
print(df.shape)

Index(['id', 'budget', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'vote_average', 'vote_count', 'return',
       'release_year', 'directed_by', 'franchise', 'produced_by',
       'produced_in', 'genres_clean'],
      dtype='object')
(45348, 21)


In [5]:
def print_sample(df):
   # Elige las 3 columnas aleatorias
   columns = np.random.choice(df.columns, 3, replace=False)  
   
   # Obtén el sample  
   sample = df[columns].sample(n=5).T.reset_index(drop=True).T

   # Obtenemos los nombres de columnas    
   column_names = df[columns].columns
   
   # Imprimimos el encabezado con los nombres de columna
   print(f"{', '.join(column_names)}\n")
   
   # Luego imprimos el sample      
   print(sample.to_string())

In [6]:
print_sample(df)

NameError: name 'df' is not defined

___
### Hora de exportar los datos antes de preparar las funciones de la API
___

In [41]:
# Aquí lo guardo como .pickle, nuestro viejo amigo
df.to_pickle('./data/datos_limpios.pickle')

In [56]:
df = pd.read_pickle('./data/datos_limpios.pickle')

In [57]:
df.columns

Index(['id', 'budget', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'vote_average', 'vote_count', 'return',
       'release_year', 'directed_by', 'franchise', 'produced_by',
       'produced_in', 'genres_clean'],
      dtype='object')

In [58]:
# Aquí hago una prueba de exportarlo al formato .feather para comparar el tamaño y velocidad
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')
df['runtime'] = pd.to_numeric(df['runtime'], errors='coerce')
df['vote_average'] = pd.to_numeric(df['vote_average'], errors='coerce')
df['vote_count'] = pd.to_numeric(df['vote_count'], errors='coerce')  # Convert 'vote_count' column to numeric
df.reset_index(inplace=True)
df.to_feather('./data/datos_limpios.feather')

___
si empiezas desde aquí acuérdate de hacer imports del principio del notebook

### Comparando el peso de los archivos, nos quisimos quedar con .feather
### pero después no logramos a poder usarlo en RENDER, así que nada...

In [15]:
df = pd.read_csv('./data/datos_limpios.csv')

In [16]:
print(df.columns)
print(df.shape)

Index(['index', 'id', 'budget', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'spoken_languages', 'status',
       'tagline', 'title', 'vote_average', 'vote_count', 'return',
       'release_year', 'directed_by', 'franchise', 'produced_by',
       'produced_in', 'genres_clean'],
      dtype='object')
(45348, 22)


In [39]:
def peliculas_duracion_version_antigua(pelicula:str):

    # Verificar que se proporcione al menos un carácter como argumento
    if len(pelicula.strip()) == 0:
        mensaje_error = {'error': 'Debe proporcionar al menos un carácter para buscar'}
        return mensaje_error
    
    # Escapar la cadena de búsqueda para evitar errores de expresión regular
    pelicula_escaped = re.escape(pelicula)
    
    # Crear una lista vacía para almacenar los resultados
    resultados = []
    
    # Filtrar las filas que contienen el texto especificado en el título
    df_filtrado = df[df['title'].str.contains(pelicula_escaped, case=False)]
    
    # Verificar si hay resultados
    if len(df_filtrado) == 0:
        mensaje_error = {'error': f'No se encontró ninguna película que contenga la palabra "{pelicula}"'}
        return mensaje_error
    
    # Iterar sobre las filas filtradas y agregar un diccionario de resultados para cada una
    for indice, fila in df_filtrado.iterrows():
        # Agregar un máximo de 10 resultados
        if len(resultados) == 10:
            break
        resultado = {'título': fila['title'], 'duración (minutos)': fila['runtime'], 'año lanzamiento': fila['release_year']}
        resultados.append(resultado)
    
    # Devolver la lista de resultados
    return resultados

In [40]:
def peliculas_duracion(pelicula:str):

    # Verificar que se proporcione al menos un carácter como argumento
    if len(pelicula.strip()) == 0:
        mensaje_error = {'error': 'Debe proporcionar al menos un carácter para buscar'}
        return mensaje_error, 400
    
    # Escapar la cadena de búsqueda para evitar errores de expresión regular
    pelicula_escaped = re.escape(pelicula)
    
    # Crear una lista vacía para almacenar los resultados
    resultados = []
    
    # Filtrar las filas que contienen el texto especificado en el título
    df_filtrado = df[df['title'].str.contains(pelicula_escaped, case=False)]
    
    # Verificar si hay resultados
    if len(df_filtrado) == 0:
        mensaje_error = {'error': f'No se encontró ninguna película que contenga la palabra "{pelicula}"'}
        return mensaje_error, 404
    
    # Convertir los valores "nan" del campo "runtime" a una cadena vacía
    df_filtrado['runtime'] = df_filtrado['runtime'].fillna('')
    
    # Iterar sobre las filas filtradas y agregar un diccionario de resultados para cada una
    for indice, fila in df_filtrado.iterrows():
        # Agregar un máximo de 10 resultados
        if len(resultados) == 10:
            break
        resultado = {'título': fila['title'], 'duración (minutos)': fila['runtime'], 'año lanzamiento': fila['release_year']}
        resultados.append(resultado)
    
    # Devolver la lista de resultados
    return resultados, 200

In [41]:
peliculas_duracion('tonto')

C:\Users\raftx\AppData\Local\Temp\ipykernel_2924\2988312141.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['runtime'] = df_filtrado['runtime'].fillna('')


([{'título': 'Harry and Tonto',
   'duración (minutos)': 115.0,
   'año lanzamiento': 1974},
  {'título': 'Los tontos y los estúpidos',
   'duración (minutos)': '',
   'año lanzamiento': 2014}],
 200)

In [12]:
lenguas = df['original_language'].value_counts()
print(lenguas.to_string())

original_language
en    32186
fr     2435
it     1528
ja     1347
de     1077
es      991
ru      822
hi      508
ko      444
zh      408
sv      383
pt      315
cn      313
fi      293
nl      247
da      224
pl      218
tr      150
cs      130
el      113
no      106
hu      100
fa       99
ta       78
th       75
he       67
sr       63
ro       57
te       45
ar       39
ml       36
xx       33
hr       29
bn       29
mr       24
et       24
is       24
tl       23
id       20
ka       18
sl       17
lv       17
uk       16
bs       14
ca       12
vi       10
bg       10
ab       10
sk        9
lt        9
ur        8
nb        6
wo        5
sh        5
ms        5
mk        5
sq        5
kn        3
eu        3
ky        3
kk        3
ku        3
bm        3
pa        2
ne        2
ps        2
iu        2
mn        2
af        2
am        2
bo        2
lo        2
gl        1
rw        1
uz        1
sm        1
eo        1
hy        1
lb        1
mt        1
ay        1
zu        

In [14]:
# @app.get('/peliculas_idioma/{idioma}')
def peliculas_idioma(idioma:str):
    '''
    Ingresa el idioma en formato abreviado conforme la norma internacional ISO 639-1. 
    Los idiomas más populares son:
        "en" corresponde a inglés (English).
        "fr" corresponde a francés (French).
        "it" corresponde a italiano (Italian).
        "ja" corresponde a japonés (Japanese).
        "de" corresponde a alemán (German).
        "es" corresponde a español (Spanish).
        "ru" corresponde a ruso (Russian).
        "hi" corresponde a hindi (Hindi).
        "ko" corresponde a coreano (Korean).
        "zh" corresponde a chino (Chinese).
        "pl" corresponde a polaco (Polish)
        "ar" corresponde a argentino viste...

    La función devolverá:
        Un diccionario {'idioma':idioma, 'cantidad':respuesta}.

    '''
    df_filtrado=df[df['original_language']==idioma]
    cantidad=len(df_filtrado)
    respuesta = {'idioma':idioma, 'cantidad':cantidad}
  
    return respuesta

In [18]:
peliculas_idioma('pl')

{'idioma': 'pl', 'cantidad': 218}

In [19]:
def get_director(nombre_director: str):
    """
    Ingresar el nombre de un director incluído en el dataset.<br>

    Devuelve el éxito del mismo medido a través del retorno.<br>

    Además devuelve el nombre de cada película con su fecha de lanzamiento, retorno individual,
    costo y ganancia de la misma en formato lista.
    
    """
    # Filtramos el DataFrame por el nombre del director ingresado
    pelis_del_director = df[df['directed_by'].str.contains(nombre_director, case=False, na=False)]

    if pelis_del_director.empty:
        return f"No se encontraron películas dirigidas por {nombre_director}."

    # Limpiamos los valores 'N/A' y reemplazamos los valores 'inf' y 'nan' en el DataFrame
    pelis_del_director = pelis_del_director.replace({'N/A': np.nan, np.inf: np.nan})

    # Calculamos el promedio de éxito de las películas del director sin considerar los valores NaN
    exito_director = pelis_del_director['return'].mean(skipna=True)

    # Limitamos la cantidad de películas a devolver
    max_movies = 10  # Se puede ajustar este valor según tus necesidades
    pelis_del_director = pelis_del_director.head(max_movies)

    # Crea una lista de películas dirigidas por el director
    lista_pelis = pelis_del_director[['title', 'release_date', 'return', 'budget', 'revenue']].to_dict(orient='records')

    return {'director_success': exito_director, 'lista_peliculas': lista_pelis}

In [33]:
get_director('Grundfest')

{'director_success': 0.0,
 'lista_peliculas': [{'title': "Richard Pryor: I Ain't Dead Yet, #*%$#@!!",
   'release_date': '2003-01-01',
   'return': 0.0,
   'budget': 0,
   'revenue': 0.0}]}

In [23]:
df[df['title'] == 'The Burkittsville 7']

,index,id,budget,original_language,overview,popularity,release_date,revenue,runtime,spoken_languages,...,title,vote_average,vote_count,return,release_year,directed_by,franchise,produced_by,produced_in,genres_clean
45341,45385,289923,0,en,A film archivist revisits the story of Rustin ...,0.38645,2000-10-03,0.0,30.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,The Burkittsville 7,7.0,1.0,0.0,2000,['Ben Rock'],NaN,['Neptune Salad Entertainment' 'Pirie Producti...,['United States of America'],['Horror']


In [30]:
#@app.get('/peliculas_duracion/{pelicula}')
def peliculas_duracion_sin_re(pelicula: str):
    """
    Ingresa un título o parte del título y la API devolverá<br>
    ese título con su duración y año de estreno (o varios)<br>
    """

    # Verificar que se proporcione al menos un carácter como argumento
    if len(pelicula.strip()) == 0:
        mensaje_error = {'error': 'Debe proporcionar al menos un carácter para buscar'}
        return mensaje_error

    # Escapar los caracteres especiales en la cadena de búsqueda para evitar errores
    pelicula_escaped = pelicula.replace('*', r'\*').replace('\'', r'\'')
    
    # Crear una lista vacía para almacenar los resultados
    resultados = []
    
    # Filtrar las filas que contienen el texto especificado en el título
    df_filtrado = df[df['title'].str.contains(pelicula_escaped, case=False)]
    
    # Verificar si hay resultados
    if len(df_filtrado) == 0:
        mensaje_error = {'error': f'No se encontró ninguna película que contenga la palabra "{pelicula}"'}
        return mensaje_error
    
    # Iterar sobre las filas filtradas y agregar un diccionario de resultados para cada una
    for indice, fila in df_filtrado.iterrows():
        # Agregar un máximo de 10 resultados
        if len(resultados) == 10:
            break
        resultado = {'titulo': fila['title'], 'duracion (minutos)': fila['runtime'], 'año lanzamiento': fila['release_year']}
        resultados.append(resultado)
    
    # Devolver la lista de resultados
    return resultados

In [31]:
peliculas_duracion_sin_re("7")

[{'titulo': 'Se7en', 'duracion (minutos)': 127.0, 'año lanzamiento': 1995},
 {'titulo': 'To Gillian on Her 37th Birthday',
  'duracion (minutos)': 93.0,
  'año lanzamiento': 1996},
 {'titulo': '24 7: Twenty Four Seven',
  'duracion (minutos)': 96.0,
  'año lanzamiento': 1997},
 {'titulo': 'Airport 1975',
  'duracion (minutos)': 107.0,
  'año lanzamiento': 1974},
 {'titulo': "Airport '77",
  'duracion (minutos)': 114.0,
  'año lanzamiento': 1977},
 {'titulo': "The Concorde... Airport '79",
  'duracion (minutos)': 123.0,
  'año lanzamiento': 1979},
 {'titulo': '47 Samurai',
  'duracion (minutos)': 207.0,
  'año lanzamiento': 1962},
 {'titulo': 'The 7th Voyage of Sinbad',
  'duracion (minutos)': 88.0,
  'año lanzamiento': 1958},
 {'titulo': 'Stalag 17', 'duracion (minutos)': 120.0, 'año lanzamiento': 1953},
 {'titulo': 'U-571', 'duracion (minutos)': 116.0, 'año lanzamiento': 2000}]

In [32]:
df[df['title'].str.contains(r'\*', case=False)]

,index,id,budget,original_language,overview,popularity,release_date,revenue,runtime,spoken_languages,...,title,vote_average,vote_count,return,release_year,directed_by,franchise,produced_by,produced_in,genres_clean
3685,3689,11624,2000000,en,"A collection of seven vignettes, which each ad...",5.378481,1972-08-06,18016290.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,Everything You Always Wanted to Know About Sex...,6.6,218.0,9.008145,1972,['Woody Allen'],NaN,['United Artists'],['United States of America'],['Comedy']
4340,4345,14911,0,en,"Lt. Col. Iceal ""Ham"" Hambleton is a weapons co...",9.179655,1988-07-21,3184348.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,Bat*21,5.9,42.0,0.000000,1988,['Peter Markle'],NaN,['TriStar Pictures' 'Vision PDG'],['United States of America'],['Action' 'Drama' 'War']
4925,4931,651,3500000,en,"One of the world's most acclaimed comedies, MA...",7.701349,1970-01-24,81600000.0,116.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,M*A*S*H,6.9,238.0,23.314286,1970,['Robert Altman'],NaN,['Twentieth Century Fox Film Corporation' 'Asp...,['United States of America'],['Comedy' 'Drama' 'War']
9257,9269,20536,0,en,"In 1961, Roger Maris and Mickey Mantle played ...",2.077090,2001-04-28,0.0,129.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,61*,6.3,34.0,0.000000,2001,['Billy Crystal'],NaN,['61* Productions Inc.'],['United States of America'],['Drama' 'History']
9500,9512,8357,0,en,Amanda (Marlee Maitlin) is a divorced woman wh...,7.631049,2004-04-23,10000000.0,109.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,What the #$*! Do We (K)now!?,5.8,34.0,0.000000,2004,['William Arntz' 'Betsy Chasse'],What the Bleep! Collection,['Lord of the Wind'],['United States of America'],['Documentary']
12635,12651,13636,0,en,"In America, we define ourselves in the superla...",6.108182,2008-05-30,307811.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,Bigger Stronger Faster*,7.0,41.0,0.000000,2008,['Chris Bell'],NaN,['BSF Film'],['United States of America'],['Documentary']
15111,15134,36325,0,en,Juan “Accidentes” Dominguez is on his biggest ...,0.502688,2009-10-09,0.0,87.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",...,Bananas!*,5.6,5.0,0.000000,2009,['Fredrik Gertten'],Bananas!* Collection,['Magic Hour Films ApS'],['United States of America' 'Nicaragua' 'France'],['Documentary']
18904,18930,84177,0,en,Dole Food Company wages a campaign to prevent ...,0.296487,2011-11-19,0.0,90.0,"[{'iso_639_1': 'sv', 'name': 'svenska'}]",...,Big Boys Gone Bananas!*,7.2,7.0,0.000000,2011,['Fredrik Gertten'],Bananas!* Collection,['WG Film'],['Sweden' 'United States of America'],['Documentary']
23858,23886,254446,0,pt,A look at the high-octane global live music in...,0.071652,2013-03-22,0.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,Who the F**K Is Arthur Fogel,5.0,1.0,0.000000,2013,['Ron Chapman'],NaN,[],['Canada'],['Documentary']
26444,26487,35405,0,en,"Two CIA bunglers (Donald Sutherland, Elliott G...",0.774252,1974-09-05,0.0,87.0,"[{'iso_639_1': 'en', 'name': 'English'}]",...,S*P*Y*S,5.9,5.0,0.000000,1974,['Irvin Kershner'],NaN,['Twentieth Century Fox Film Corporation' 'Ame...,['United States of America'],['Action' 'Comedy']


In [34]:
peliculas_duracion_sin_re('What the #$*! Do We (K)now!?')

C:\Users\raftx\AppData\Local\Temp\ipykernel_2924\1968515713.py:20: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_filtrado = df[df['title'].str.contains(pelicula_escaped, case=False)]


{'error': 'No se encontró ninguna película que contenga la palabra "What the #$*! Do We (K)now!?"'}